
# Excel（.xlsm）VBA 一括エクスポート **＋** Azure OpenAI要約 **＋** 最終Markdown生成（プロキシ対応）

このノートブックは、次の処理を**ステップごと**に実行します：

1. **高速エクスポート**：.xlsm 内のVBAを `.bas/.cls/.frm(+.frx)` に一括出力（UserForm以外は高速なテキスト抽出）。  
2. **生成AIの準備**：.env から **プロキシ/キー/エンドポイント/バージョン** を読み込み、Azure OpenAI に接続。  
3. **要約（ファイル単位）**：出力フォルダ内の各ファイルを順に読み、**機能・主要処理・イベント・外部依存・改善点**などを日本語で要約。  
4. **関係性の推定**：全モジュールの一覧と主要なプロシージャ名から、**機能関係のシーケンス図（Mermaid）** を生成。  
5. **最終レポート生成**：すべての要約とシーケンス図をまとめた **1つの .md** ファイルを作成。

---

## 必要要件
- Windows + Excelデスクトップ版
- Python 3.x
- `pywin32`, `python-dotenv`, `openai`（Azure SDK）  
- Excel: **[ファイル] → [オプション] → [セキュリティ センター] → [セキュリティ センターの設定] → [マクロの設定] → 「VBA プロジェクト オブジェクト モデルへの信頼アクセスを許可する」** をオン

> 機密/社内コードの外部送信可否にご注意ください（必要に応じて匿名化・マスキング）。


In [ ]:

# ==============================
# 設定
# ==============================

# 対象のExcelブック（.xlsm）のフルパス
# 例: r"C:\work\S-SW-350-11_RO検査装置.xlsm"
workbook_path = r"S-SW-350-11_RO検査装置.xlsm"

# 出力先（ローカルSSD推奨）
output_dir = r".\vba_export_ai"

# 最終レポート（Markdown）のファイル名
final_report_name = "vba_project_summary.md"

# READMEも生成するか
make_readme = True

# ---- ここから下は通常は変更不要 ----
import os, sys, traceback, time, re, csv, json, glob
from pathlib import Path
Path(output_dir).mkdir(parents=True, exist_ok=True)
print("Workbook Path:", os.path.abspath(workbook_path))
print("Output Dir   :", Path(output_dir).resolve())


In [ ]:

# 必要に応じてインストールしてください:
# !pip install pywin32
# !pip install python-dotenv
# !pip install openai

try:
    import win32com.client as win32
except Exception:
    print("pywin32 の読み込みに失敗しました。'pip install pywin32' 実行後に再試行してください。")
    raise


In [ ]:

# ========== 高速エクスポート ==========

CT_STD, CT_CLASS, CT_FORM, CT_DOC = 1, 2, 3, 100
EXT_MAP = {CT_STD: ".bas", CT_CLASS: ".cls", CT_FORM: ".frm", CT_DOC: ".cls"}

def safe_filename(name: str) -> str:
    return re.sub(r'[\\/:*?"<>|]', "_", name.strip())

def write_text(path, text, encoding="utf-8"):
    with open(path, "w", encoding=encoding, errors="ignore") as f:
        f.write(text)

def export_component_fast(comp, out_dir, use_export_for_forms=True):
    """
    - UserForm(Type==3) は Export() で .frm + .frx を生成
    - それ以外は CodeModule.Lines で高速テキスト出力
    """
    ctype = comp.Type
    name = comp.Name
    ext = EXT_MAP.get(ctype, ".cls")
    fname = safe_filename(f"{name}{ext}")
    fpath = os.path.join(out_dir, fname)

    stem, ext_only = os.path.splitext(fname)
    idx = 1
    while os.path.exists(fpath):
        fname = f"{stem}_{idx}{ext_only}"
        fpath = os.path.join(out_dir, fname)
        idx += 1

    if ctype == CT_FORM and use_export_for_forms:
        comp.Export(os.path.abspath(fpath))
        return fpath
    else:
        cm = comp.CodeModule
        total_lines = cm.CountOfLines
        code = cm.Lines(1, total_lines) if total_lines > 0 else ""
        write_text(os.path.abspath(fpath), code)
        return fpath

def export_all_vba_fast(workbook_path: str, out_dir: str, verbose=True):
    t0 = time.time()
    excel = win32.DispatchEx("Excel.Application")
    excel.Visible = False
    try:
        excel.AutomationSecurity = 3  # msoAutomationSecurityForceDisable
    except Exception:
        pass
    try:
        excel.DisplayAlerts = False
    except Exception:
        pass
    try:
        excel.AskToUpdateLinks = False
    except Exception:
        pass

    wb = None
    exported = []
    try:
        wb = excel.Workbooks.Open(os.path.abspath(workbook_path), UpdateLinks=0, ReadOnly=True)
        comps = wb.VBProject.VBComponents
        n = comps.Count
        if verbose:
            print(f"VBコンポーネント数: {n}")

        for i in range(1, n+1):
            comp = comps.Item(i)
            name = comp.Name
            ctype = comp.Type
            label = {1:"StdModule",2:"Class",3:"Form",100:"Document"}.get(ctype, str(ctype))

            t1 = time.time()
            out_path = export_component_fast(comp, out_dir, use_export_for_forms=True)
            dt = time.time() - t1

            print(f"[{i}/{n}] {name} ({label}) -> {out_path}  [{dt:.2f}s]")
            exported.append((name, ctype, out_path))

        # マニフェスト
        manifest_csv = os.path.join(out_dir, "manifest.csv")
        with open(manifest_csv, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(["ComponentName", "Type", "ExportPath"])
            w.writerows(exported)

        manifest_json = os.path.join(out_dir, "manifest.json")
        with open(manifest_json, "w", encoding="utf-8") as f:
            json.dump([{"name": n, "type": t, "path": p} for n, t, p in exported], f, ensure_ascii=False, indent=2)

        print(f"\nマニフェスト出力: {manifest_csv}, {manifest_json}")
        print(f"総所要時間: {time.time() - t0:.2f}s")
        return exported

    finally:
        if wb is not None:
            wb.Close(SaveChanges=False)
        excel.Quit()


In [ ]:

# 実行
results = export_all_vba_fast(workbook_path, output_dir, verbose=True)

print("\nエクスポート結果サマリ:")
for name, ctype, path in results:
    print(f" - {name} (Type={ctype}) -> {path}")


In [ ]:

# README を出力
if make_readme:
    readme_path = os.path.join(output_dir, "README.txt")
    with open(readme_path, "w", encoding="utf-8") as f:
        f.write("\u3010VBE \u3078\u306e\u518d\u30a4\u30f3\u30dd\u30fc\u30c8\u624b\u9806\u3011\n1) Excel\uff08.xlsm\uff09\u3092\u958b\u304f -> Alt+F11 \u3067VBE\u3092\u958b\u304f\n2) \u5bfe\u8c61\u30d7\u30ed\u30b8\u30a7\u30af\u30c8\u3092\u53f3\u30af\u30ea\u30c3\u30af -> [\u30d5\u30a1\u30a4\u30eb\u306e\u30a4\u30f3\u30dd\u30fc\u30c8]\n3) .bas / .cls / .frm \u3092\u9078\u629e\u3057\u3066\u30a4\u30f3\u30dd\u30fc\u30c8\u3059\u308b\uff08\u30d5\u30a9\u30fc\u30e0\u306f .frx \u3082\u5fc5\u8981\uff09\n\n\u3010\u6ce8\u610f\u3011\n- \u30c9\u30ad\u30e5\u30e1\u30f3\u30c8\u30e2\u30b8\u30e5\u30fc\u30eb\uff08ThisWorkbook\u3001Sheet1 \u306a\u3069\uff09\u306f .cls \u3068\u3057\u3066\u51fa\u529b\u3055\u308c\u307e\u3059\u3002\n- \u6a5f\u5bc6\u60c5\u5831\u306e\u5916\u90e8\u9001\u4fe1\u306b\u6ce8\u610f\u3057\u3066\u304f\u3060\u3055\u3044\u3002\n")
    print("README を出力しました:", readme_path)



## Azure OpenAI 準備（.env とプロキシ）

同ディレクトリに `.env` を置いてください。例：

```
HTTP_PROXY=http://XXX.XX.X.XX:XXXX
HTTPS_PROXY=http://XXX.XX.X.XX:XXXX
API_KEY=3pJrXXXXXXXXXXXXXXXX
API_VERSION=2025-01-01-preview
AZURE_ENDPOINT=https://XXXXXX.openai.azure.com/
```

> モデルのデプロイ名は環境に合わせて設定してください（例：`gpt-4o-us-MEM-DX-openai-001`）。


In [ ]:

# 必要に応じて:
# !pip install python-dotenv
# !pip install openai

from dotenv import load_dotenv
from openai import AzureOpenAI
import os

load_dotenv()  # .env 読み込み

# プロキシ（必要な場合のみ）
os.environ["http_proxy"]  = os.getenv("HTTP_PROXY", "")
os.environ["https_proxy"] = os.getenv("HTTPS_PROXY", "")

# Azure OpenAI クライアント
client = AzureOpenAI(
    api_key       = os.getenv("API_KEY"),
    api_version   = os.getenv("API_VERSION"),
    azure_endpoint= os.getenv("AZURE_ENDPOINT")
)

# 動作確認
resp = client.chat.completions.create(
    model="gpt-4o-us-MEM-DX-openai-001",  # ← 貴環境のデプロイ名に変更
    messages=[{"role": "user", "content": "接続テスト：1行で応答してください。"}],
    temperature=0
)
print(resp.choices[0].message.content)


In [ ]:

def read_text_safely(path, max_chars=30000):
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()[:max_chars]
    except Exception as e:
        return f"<<読み込みエラー: {e}>>"

def extract_procedures(code_text):
    """
    簡易的に Sub/Function/Property の宣言行を列挙（正確性よりも手軽さ重視）
    """
    proc_pat = re.compile(r'^\s*(Public|Private)?\s*(Sub|Function|Property\s+Get|Property\s+Let|Property\s+Set)\s+([A-Za-z0-9_]+)', re.IGNORECASE|re.MULTILINE)
    procs = []
    for m in proc_pat.finditer(code_text):
        name = m.group(3)
        kind = m.group(2)
        procs.append(f"{kind} {name}")
    return procs



## ステップ1：モジュール一覧の作成
- 出力フォルダ内の `.bas/.cls/.frm` を列挙し、先頭数万文字を読み込みます。  
- さらに簡易的に **Sub/Function/Property** の宣言行を抽出し、後続の関係性推定に使います。


In [ ]:

from pathlib import Path
import glob

vba_files = []
for ext in ("*.bas", "*.cls", "*.frm"):
    vba_files.extend(glob.glob(os.path.join(output_dir, ext)))

modules_info = []  # [{name, path, procedures, preview}, ...]
for fp in sorted(vba_files):
    name = os.path.splitext(os.path.basename(fp))[0]
    code_text = read_text_safely(fp, max_chars=60000)  # 大きいほど精度は上がるがトークンに注意
    procs = extract_procedures(code_text)
    modules_info.append({
        "name": name,
        "path": fp,
        "procedures": procs[:200],  # 上限をかける
        "preview": code_text[:2000],  # 関係性推定には冒頭だけ
    })

print("モジュール数:", len(modules_info))
for m in modules_info[:5]:
    print("-", m["name"], f"(proc: {len(m['procedures'])})")



## ステップ2：各ファイルの要約（ステップごとの明確な目的）
各ファイルに対して以下の**目的**を明示したプロンプトで要約します：

- 目的：コードの**役割・主要ロジック・入出力・イベント・外部依存・危険箇所・改善点**を日本語で、箇条書き中心で簡潔に。  
- 制約：**事実の推測を避ける**、不明点は「不明」と明記。  
- 形式：見出し＋箇条書き。

> 大きなファイルは先頭数万文字に制限して送信します（トークン過多回避）。


In [ ]:

per_file_summaries = []  # [{name, path, summary_text}, ...]

summary_dir = os.path.join(output_dir, "_summaries")
Path(summary_dir).mkdir(parents=True, exist_ok=True)

for m in modules_info:
    name = m["name"]
    path = m["path"]
    text = read_text_safely(path, max_chars=30000)

    messages = [
        {"role": "system", "content": "あなたはVBAコードのレビューアです。事実に基づき日本語で簡潔にまとめます。"},
        {"role": "user", "content": f"""目的:
- このVBAモジュールの役割/主要ロジック/入出力/イベント/外部依存/危険箇所/改善点を日本語で簡潔に、箇条書き中心で要約してください。
- 不明な点は推測せず「不明」と書いてください。
- 出力は以下の形式で:
# {name} の要約
- 役割:
- 主要ロジック:
- 入出力:
- イベント:
- 外部依存:
- 危険箇所:
- 改善点:

--- VBAコード（先頭抜粋/最大30,000字） ---
{text}
"""},
    ]

    try:
        res = client.chat.completions.create(
            model="gpt-4o-us-MEM-DX-openai-001",  # ← 環境のデプロイ名に合わせる
            messages=messages,
            temperature=0
        )
        summary = res.choices[0].message.content
    except Exception as e:
        summary = f"<<要約エラー: {e}>>"

    per_file_summaries.append({"name": name, "path": path, "summary_text": summary})

    out_txt = os.path.join(summary_dir, f"{name}.summary.md")
    with open(out_txt, "w", encoding="utf-8") as f:
        f.write(summary)
    print("要約出力:", out_txt)



## ステップ3：モジュール間の関係性を推定し、**Mermaidシーケンス図**を生成
- 入力：モジュール名、抽出したプロシージャ名のリスト、各要約の見出し（必要に応じて短縮）。  
- 出力：`mermaid` 記法の **sequenceDiagram**。

> 実コード全体を渡さずに、手がかり情報から相互呼び出し・イベント連鎖を**推定**します。不確実な箇所は「仮」と注記させます。


In [ ]:

# Mermaid 図の生成用に、情報を圧縮して渡す
module_briefs = []
for m in modules_info:
    name = m["name"]
    procs = m["procedures"][:30]
    module_briefs.append({"name": name, "procedures": procs})

# LLMへプロンプト
mermaid_prompt = {
    "role": "user",
    "content": f"""目的:
- 下記のモジュール一覧と、その中の代表的なプロシージャ名から、想定される呼び出し関係・イベントの流れを推定し、Mermaidの sequenceDiagram を出力してください。
- 不確実な関係は「(仮)」と注記してください。
- できるだけ主要な処理の流れが一望できるよう、登場ノードは過不足なく整理してください。

出力形式:
```mermaid
sequenceDiagram
  %% ここに図を書く
```

モジュール情報(要約):
{json.dumps(module_briefs, ensure_ascii=False, indent=2)}
"""
}

try:
    res = client.chat.completions.create(
        model="gpt-4o-us-MEM-DX-openai-001",
        messages=[
            {"role": "system", "content": "あなたはソフトウェアアーキテクトです。与えられた手がかりから合理的に流れを推定し、Mermaidのシーケンス図を日本語の注釈付きで作成します。"},
            mermaid_prompt
        ],
        temperature=0
    )
    mermaid_block = res.choices[0].message.content
except Exception as e:
    mermaid_block = f"<<Mermaid生成エラー: {e}>>"

print(mermaid_block[:800], "...\n\n(全文は次のセルで最終MDに書き込み)")



## ステップ4：**最終Markdown**の組み立てと保存
- 全モジュールの要約を連結し、先頭に **目次** と **Mermaidシーケンス図** を配置します。  
- 出力先：`output_dir` / `final_report_name`


In [ ]:

# 目次・ヘッダ組み立て
toc_lines = [f"- [{m['name']}](#{m['name'].lower()})" for m in per_file_summaries]

header = [
    "# VBA プロジェクト総合レポート",
    "",
    "## 目次",
    "\n".join(toc_lines),
    "",
    "## シーケンス図（Mermaid推定）",
    mermaid_block,
    "",
    "---",
    "## 各モジュールの要約",
    ""
]

body_parts = []
for item in per_file_summaries:
    body_parts.append(item["summary_text"])
    body_parts.append("\n---\n")

final_md = "\n".join(header + body_parts)

final_md_path = os.path.join(output_dir, final_report_name)
with open(final_md_path, "w", encoding="utf-8") as f:
    f.write(final_md)

print("最終Markdownを出力しました:", final_md_path)



> **補足**  
> - Mermaid図は手がかり情報からの**推定**です。必要に応じて手動で調整してください。  
> - ファイルが巨大/多数の場合は、`read_text_safely` の `max_chars` を調節したり、対象ファイルを限定するなどして精度とコストのバランスを取ってください。  
> - さらに精度を上げるには、**実コード全体**や**依存関係の抽出（`Declare PtrSafe`, `CreateObject`, `WithEvents`, `Application.Run` など）**を追加で解析して、LLMに渡すヒントを厚くするのがおすすめです。
